In [76]:
import pandas as pd

In [77]:
#read match data

In [78]:
matches = pd.read_csv("laliga_games.csv", index_col = 0)

In [79]:
matches.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt,Season,Team
1,2024-08-18,21:30,La Liga,Matchweek 1,Sun,Away,D,1.0,1.0,Mallorca,...,Match Report,NaN,13.0,5.0,21.3,0.0,0,0,2025,Real Madrid
2,2024-08-25,17:00,La Liga,Matchweek 2,Sun,Home,W,3.0,0.0,Valladolid,...,Match Report,NaN,17.0,9.0,17.2,1.0,0,0,2025,Real Madrid
3,2024-08-29,20:30,La Liga,Matchweek 3,Thu,Away,D,1.0,1.0,Las Palmas,...,Match Report,NaN,24.0,7.0,18.0,1.0,1,1,2025,Real Madrid
4,2024-09-01,21:30,La Liga,Matchweek 4,Sun,Home,W,2.0,0.0,Betis,...,Match Report,NaN,21.0,6.0,16.3,0.0,1,1,2025,Real Madrid
5,2024-09-14,21:00,La Liga,Matchweek 5,Sat,Away,W,2.0,0.0,Real Sociedad,...,Match Report,NaN,14.0,4.0,19.4,1.0,2,2,2025,Real Madrid


In [80]:
matches.shape

(5684, 28)

In [81]:
#how many matches per team

In [82]:
matches["Team"].value_counts()

Team
Real Madrid            285
Barcelona              285
Athletic Club          285
Atletico Madrid        284
Villarreal             284
Valencia               284
Real Sociedad          284
Real Betis             284
Celta Vigo             284
Getafe                 284
Sevilla                284
Alaves                 246
Espanyol               208
Osasuna                208
Levante                190
Mallorca               171
Girona                 170
Valladolid             170
Rayo Vallecano         170
Eibar                  152
Granada                152
Cadiz                  152
Leganes                132
Elche                  114
Las Palmas              94
Almeria                 76
Huesca                  76
Deportivo La Coruna     38
Malaga                  38
Name: count, dtype: int64

In [83]:
#look at data types

In [84]:
matches.dtypes

Date              object
Time              object
Comp              object
Round             object
Day               object
Venue             object
Result            object
GF               float64
GA               float64
Opponent          object
xG               float64
xGA              float64
Poss             float64
Attendance       float64
Captain           object
Formation         object
Opp Formation     object
Referee           object
Match Report      object
Notes            float64
Sh               float64
SoT              float64
Dist             float64
FK               float64
PK                 int64
PKatt              int64
Season             int64
Team              object
dtype: object

In [85]:
#convert existing column to date time

In [86]:
matches["Date"] = pd.to_datetime(matches["Date"])

In [87]:
#conver home away to numeric. 0 away, 1 home

In [88]:
matches["venue_code"] = matches["Venue"].astype("category").cat.codes

In [89]:
#code for each opponent squad

In [90]:
matches["opp_code"] = matches["Opponent"].astype("category").cat.codes

In [91]:
#look at time of games and remove colon and minutes and just have hour

In [92]:
matches["hour"] = matches["Time"].str.replace(":.+", "", regex = True).astype("int")

In [93]:
#code each day of the week

In [94]:
matches["day_code"] = matches["Date"].dt.dayofweek

In [95]:
#win is a 1 and loss or draw is a 0

In [96]:
matches["target"] = (matches["Result"] == "W").astype("int")

In [97]:
# import model for machine learning

In [98]:
from sklearn.ensemble import RandomForestClassifier

In [99]:
#initialize class

In [100]:
rf = RandomForestClassifier(n_estimators = 50, min_samples_split = 10, random_state = 1)

In [101]:
#put anything before 2022 in training set

In [102]:
train = matches[matches["Date"] < '2022-01-01']

In [103]:
test = matches[matches["Date"] > '2022-01-01']

In [104]:
#create predictor list

In [105]:
predictors = ["venue_code", "opp_code", "hour", "day_code"]

In [106]:
#fit random forest model and predict target

In [107]:
rf.fit(train[predictors], train["target"])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [108]:
#generate predictions 

In [109]:
preds = rf.predict(test[predictors])

In [110]:
#determine accuracy of predictor

In [111]:
from sklearn.metrics import accuracy_score

In [112]:
acc = accuracy_score(test["target"], preds)

In [113]:
acc

0.6092105263157894

In [114]:
#in which situations was accuracy high and low

In [115]:
combined = pd.DataFrame(dict(actual = test["target"], prediction = preds))

In [116]:
#what happened when predicting win or loss

In [117]:
pd.crosstab(index = combined["actual"], columns = combined["prediction"])

prediction,0,1
actual,,
0,1070,366
1,525,319


In [118]:
#revise accuraty metric

In [119]:
from sklearn.metrics import precision_score

In [120]:
precision_score(test["target"], preds)

0.4656934306569343

In [121]:
#improve percision by making more predictors

In [122]:
grouped_matches = matches.groupby("Team")

In [123]:
group = grouped_matches.get_group("Real Madrid")

In [124]:
group

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,FK,PK,PKatt,Season,Team,venue_code,opp_code,hour,day_code,target
1,2024-08-18,21:30,La Liga,Matchweek 1,Sun,Away,D,1.0,1.0,Mallorca,...,0.0,0,0,2025,Real Madrid,0,19,21,6,0
2,2024-08-25,17:00,La Liga,Matchweek 2,Sun,Home,W,3.0,0.0,Valladolid,...,1.0,0,0,2025,Real Madrid,1,27,17,6,1
3,2024-08-29,20:30,La Liga,Matchweek 3,Thu,Away,D,1.0,1.0,Las Palmas,...,1.0,1,1,2025,Real Madrid,0,16,20,3,0
4,2024-09-01,21:30,La Liga,Matchweek 4,Sun,Home,W,2.0,0.0,Betis,...,0.0,1,1,2025,Real Madrid,1,5,21,6,1
5,2024-09-14,21:00,La Liga,Matchweek 5,Sat,Away,W,2.0,0.0,Real Sociedad,...,1.0,2,2,2025,Real Madrid,0,24,21,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53,2018-04-28,18:30,La Liga,Matchweek 35,Sat,Home,W,2.0,1.0,Leganés,...,1.0,0,0,2018,Real Madrid,1,17,18,5,1
55,2018-05-06,20:45,La Liga,Matchweek 36,Sun,Away,D,2.0,2.0,Barcelona,...,1.0,0,0,2018,Real Madrid,0,4,20,6,0
56,2018-05-09,21:30,La Liga,Matchweek 34,Wed,Away,L,2.0,3.0,Sevilla,...,2.0,1,2,2018,Real Madrid,0,25,21,2,0
57,2018-05-12,20:45,La Liga,Matchweek 37,Sat,Home,W,6.0,0.0,Celta Vigo,...,1.0,0,0,2018,Real Madrid,1,6,20,5,1


In [125]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("Date")
    rolling_stats = group[cols].rolling(3, closed = 'left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset = new_cols)
    return group

In [126]:
cols = ["GF", "GA", "Sh", "SoT", "Dist", "PK", "PKatt"]
new_cols = [f"{c}_rolling" for c in cols]

In [127]:
new_cols

['GF_rolling',
 'GA_rolling',
 'Sh_rolling',
 'SoT_rolling',
 'Dist_rolling',
 'PK_rolling',
 'PKatt_rolling']

In [128]:
rolling_averages(group, cols, new_cols)

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,hour,day_code,target,GF_rolling,GA_rolling,Sh_rolling,SoT_rolling,Dist_rolling,PK_rolling,PKatt_rolling
7,2017-09-17,20:45,La Liga,Matchweek 4,Sun,Away,W,3.0,1.0,Real Sociedad,...,20,6,1,2.000000,1.000000,18.666667,8.000000,17.433333,0.000000,0.000000
8,2017-09-20,22:00,La Liga,Matchweek 5,Wed,Home,L,0.0,1.0,Betis,...,22,2,0,2.000000,1.333333,19.333333,7.333333,16.333333,0.000000,0.000000
9,2017-09-23,16:15,La Liga,Matchweek 6,Sat,Away,W,2.0,1.0,Alavés,...,16,5,1,1.333333,1.000000,21.000000,7.333333,15.533333,0.000000,0.000000
11,2017-10-01,20:45,La Liga,Matchweek 7,Sun,Home,W,2.0,0.0,Espanyol,...,20,6,1,1.666667,1.000000,22.000000,6.333333,16.066667,0.000000,0.000000
12,2017-10-14,16:15,La Liga,Matchweek 8,Sat,Away,W,2.0,1.0,Getafe,...,16,5,1,1.333333,0.666667,23.666667,7.000000,17.033333,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,2024-12-04,21:00,La Liga,Matchweek 19,Wed,Away,L,1.0,2.0,Athletic Club,...,21,2,0,3.000000,0.000000,15.333333,6.333333,16.133333,0.333333,0.333333
21,2024-12-07,21:00,La Liga,Matchweek 16,Sat,Away,W,3.0,0.0,Girona,...,21,5,1,2.000000,0.666667,12.333333,5.000000,16.900000,0.333333,0.666667
23,2024-12-14,21:00,La Liga,Matchweek 17,Sat,Away,D,3.0,3.0,Rayo Vallecano,...,21,5,0,2.000000,0.666667,10.333333,4.666667,18.133333,0.333333,0.666667
24,2024-12-22,16:15,La Liga,Matchweek 18,Sun,Home,W,4.0,2.0,Sevilla,...,16,6,1,2.333333,1.666667,10.666667,5.666667,19.466667,0.000000,0.333333


In [129]:
#apply this to all matches

In [130]:
matches_rolling = matches.groupby("Team").apply(lambda x: rolling_averages(x, cols, new_cols))

In [131]:
matches_rolling

Date   Time     Comp         Round  Day Venue Result   GF  \
Team                                                                            
Alaves     3  2017-09-17  12:00  La Liga   Matchweek 4  Sun  Home      L  0.0   
           4  2017-09-20  21:00  La Liga   Matchweek 5  Wed  Away      L  0.0   
           5  2017-09-23  16:15  La Liga   Matchweek 6  Sat  Home      L  1.0   
           6  2017-09-30  18:30  La Liga   Matchweek 7  Sat  Away      W  2.0   
           7  2017-10-14  18:30  La Liga   Matchweek 8  Sat  Home      L  0.0   
...                  ...    ...      ...           ...  ...   ...    ...  ...   
Villarreal 14 2024-12-01  14:00  La Liga  Matchweek 15  Sun  Home      D  2.0   
           16 2024-12-08  16:15  La Liga  Matchweek 16  Sun  Away      L  0.0   
           17 2024-12-15  18:30  La Liga  Matchweek 17  Sun  Home      L  1.0   
           18 2024-12-18  21:30  La Liga  Matchweek 12  Wed  Home      D  1.0   
           19 2024-12-22  18:30  La Liga  Matchweek 18  Sun  Away      W  5.0   

                GA        Opponent  ...  hour  day_code  target  GF_rolling  \
Team                                ...                                       
Alaves     3   3.0      Villarreal  ...    12         6       0    0.000000   
           4   1.0       La Coruña  ...    21         2       0    0.000000   
           5   2.0     Real Madrid  ...    16         5       0    0.000000   
           6   0.0         Levante  ...    18         5       1    0.333333   
           7   2.0   Real Sociedad  ...    18         5       0    1.000000   
...            ...             ...  ...   ...       ...     ...         ...   
Villarreal 14  2.0          Girona  ...    14         6       0    2.333333   
           16  2.0   Athletic Club  ...    16         6       0    2.333333   
           17  2.0           Betis  ...    18         6       0    1.333333   
           18  1.0  Rayo Vallecano  ...    21         2       0    1.000000   
           19  2.0         Leganés  ...    18         6       1    0.666667   

              GA_rolling Sh_rolling SoT_rolling Dist_rolling PK_rolling  \
Team                                                                      
Alaves     3    1.333333   8.000000    3.000000    20.066667   0.000000   
           4    2.000000  10.666667    2.333333    20.533333   0.000000   
           5    1.666667   9.666667    2.666667    20.200000   0.000000   
           6    2.000000  10.666667    2.666667    21.233333   0.000000   
           7    1.000000   8.000000    3.000000    17.633333   0.000000   
...                  ...        ...         ...          ...        ...   
Villarreal 14   1.000000  16.000000    5.666667    17.166667   0.666667   
           16   1.333333  13.333333    4.000000    16.400000   0.666667   
           17   2.000000  13.000000    3.666667    16.166667   0.333333   
           18   2.000000  13.666667    2.666667    15.533333   0.000000   
           19   1.666667  14.000000    3.000000    16.366667   0.000000   

               PKatt_rolling  
Team                          
Alaves     3        0.333333  
           4        0.000000  
           5        0.000000  
           6        0.000000  
           7        0.000000  
...                      ...  
Villarreal 14       0.666667  
           16       0.666667  
           17       0.333333  
           18       0.000000  
           19       0.000000  

[5591 rows x 40 columns]

In [132]:
matches_rolling = matches_rolling.droplevel('Team')

In [133]:
matches_rolling

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,hour,day_code,target,GF_rolling,GA_rolling,Sh_rolling,SoT_rolling,Dist_rolling,PK_rolling,PKatt_rolling
3,2017-09-17,12:00,La Liga,Matchweek 4,Sun,Home,L,0.0,3.0,Villarreal,...,12,6,0,0.000000,1.333333,8.000000,3.000000,20.066667,0.000000,0.333333
4,2017-09-20,21:00,La Liga,Matchweek 5,Wed,Away,L,0.0,1.0,La Coruña,...,21,2,0,0.000000,2.000000,10.666667,2.333333,20.533333,0.000000,0.000000
5,2017-09-23,16:15,La Liga,Matchweek 6,Sat,Home,L,1.0,2.0,Real Madrid,...,16,5,0,0.000000,1.666667,9.666667,2.666667,20.200000,0.000000,0.000000
6,2017-09-30,18:30,La Liga,Matchweek 7,Sat,Away,W,2.0,0.0,Levante,...,18,5,1,0.333333,2.000000,10.666667,2.666667,21.233333,0.000000,0.000000
7,2017-10-14,18:30,La Liga,Matchweek 8,Sat,Home,L,0.0,2.0,Real Sociedad,...,18,5,0,1.000000,1.000000,8.000000,3.000000,17.633333,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14,2024-12-01,14:00,La Liga,Matchweek 15,Sun,Home,D,2.0,2.0,Girona,...,14,6,0,2.333333,1.000000,16.000000,5.666667,17.166667,0.666667,0.666667
16,2024-12-08,16:15,La Liga,Matchweek 16,Sun,Away,L,0.0,2.0,Athletic Club,...,16,6,0,2.333333,1.333333,13.333333,4.000000,16.400000,0.666667,0.666667
17,2024-12-15,18:30,La Liga,Matchweek 17,Sun,Home,L,1.0,2.0,Betis,...,18,6,0,1.333333,2.000000,13.000000,3.666667,16.166667,0.333333,0.333333
18,2024-12-18,21:30,La Liga,Matchweek 12,Wed,Home,D,1.0,1.0,Rayo Vallecano,...,21,2,0,1.000000,2.000000,13.666667,2.666667,15.533333,0.000000,0.000000


In [134]:
#unique values for each index

In [135]:
matches_rolling.index = range(matches_rolling.shape[0])

In [136]:
#make new predictions

In [137]:
def make_predictions(data, predictors):
    train = data[data["Date"] < '2022-01-01']
    test = data[data["Date"] > '2022-01-01']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual = test["target"], predicted = preds), index = test.index)
    precision = precision_score(test["target"], preds)
    return combined, precision

In [138]:
combined, precision = make_predictions(matches_rolling, predictors + new_cols)

In [139]:
precision

0.5570469798657718

In [140]:
combined

,actual,predicted
167,0,0
168,0,0
169,0,0
170,0,0
171,0,0
...,...,...
5586,0,1
5587,0,0
5588,0,1
5589,0,0


In [141]:
combined = combined.merge(matches_rolling[["Date", "Team", "Opponent", "Result"]], left_index = True, right_index = True)

In [142]:
combined

,actual,predicted,Date,Team,Opponent,Result
167,0,0,2022-01-02,Alaves,Real Sociedad,D
168,0,0,2022-01-09,Alaves,Athletic Club,D
169,0,0,2022-01-18,Alaves,Betis,L
170,0,0,2022-01-23,Alaves,Barcelona,L
171,0,0,2022-02-05,Alaves,Elche,L
...,...,...,...,...,...,...
5586,0,1,2024-12-01,Villarreal,Girona,D
5587,0,0,2024-12-08,Villarreal,Athletic Club,L
5588,0,1,2024-12-15,Villarreal,Betis,L
5589,0,0,2024-12-18,Villarreal,Rayo Vallecano,D


In [ ]:
#combine home and away perdictions.

In [144]:
class MissingDict(dict):
    __missing__ = lambda self, key: key

map_values = {
    "Alavés" : "Alaves",
    "Cádiz" : "Cadiz",
    "Atlético Madrid" : "Atletico Madrid",
    "Betis" : "Real Betis",
    "Málaga" : "Malaga",
    "Leganés" : "Leganes",
    "La Coruña" : "Deportivo La Coruna",
    "Alméria" : "Almeria",
    
    
}
mapping = MissingDict(**map_values)

In [145]:
mapping["La Coruña"]

'Deportivo La Coruna'

In [146]:
combined["new_team"] = combined["Team"].map(mapping)

In [147]:
combined

,actual,predicted,Date,Team,Opponent,Result,new_team
167,0,0,2022-01-02,Alaves,Real Sociedad,D,Alaves
168,0,0,2022-01-09,Alaves,Athletic Club,D,Alaves
169,0,0,2022-01-18,Alaves,Betis,L,Alaves
170,0,0,2022-01-23,Alaves,Barcelona,L,Alaves
171,0,0,2022-02-05,Alaves,Elche,L,Alaves
...,...,...,...,...,...,...,...
5586,0,1,2024-12-01,Villarreal,Girona,D,Villarreal
5587,0,0,2024-12-08,Villarreal,Athletic Club,L,Villarreal
5588,0,1,2024-12-15,Villarreal,Betis,L,Villarreal
5589,0,0,2024-12-18,Villarreal,Rayo Vallecano,D,Villarreal


In [148]:
merged = combined.merge(combined, left_on=["Date", "new_team"], right_on=["Date", "Opponent"])

In [149]:
merged

,actual_x,predicted_x,Date,Team_x,Opponent_x,Result_x,new_team_x,actual_y,predicted_y,Team_y,Opponent_y,Result_y,new_team_y
0,1,0,2022-01-03,Athletic Club,Osasuna,W,Athletic Club,0,0,Osasuna,Athletic Club,L,Osasuna
1,0,0,2022-01-09,Athletic Club,Alavés,D,Athletic Club,0,0,Alaves,Athletic Club,D,Alaves
2,1,0,2022-01-23,Athletic Club,Rayo Vallecano,W,Athletic Club,0,0,Rayo Vallecano,Athletic Club,L,Rayo Vallecano
3,1,1,2022-02-07,Athletic Club,Espanyol,W,Athletic Club,0,0,Espanyol,Athletic Club,L,Espanyol
4,0,0,2022-02-14,Athletic Club,Mallorca,L,Athletic Club,1,0,Mallorca,Athletic Club,W,Mallorca
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1773,0,1,2024-12-01,Villarreal,Girona,D,Villarreal,0,0,Girona,Villarreal,D,Girona
1774,0,0,2024-12-08,Villarreal,Athletic Club,L,Villarreal,1,0,Athletic Club,Villarreal,W,Athletic Club
1775,0,1,2024-12-15,Villarreal,Betis,L,Villarreal,1,0,Real Betis,Villarreal,W,Real Betis
1776,0,0,2024-12-18,Villarreal,Rayo Vallecano,D,Villarreal,0,0,Rayo Vallecano,Villarreal,D,Rayo Vallecano


In [150]:
merged[(merged["predicted_x"] == 1) & (merged["predicted_y"] ==0)]["actual_x"].value_counts()

actual_x
1    188
0    155
Name: count, dtype: int64